In [19]:
import pandas as pd
from openai import OpenAI
from pinecone import Pinecone
from keys.keys import OPENAI_API_KEY, PINECONE_API_KEY, INDEX_NAME
from functions.query_index import query_index
from functions.read_docs import read_docs
from functions.generate_answer import generate_answer
# from functions.query import query

In [20]:
client = OpenAI(api_key=OPENAI_API_KEY)
model = 'text-embedding-ada-002'

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

In [21]:
file_name = "/Users/tashatanarugsachock/Desktop/shopperbot-llm/dataset/dataset.xlsx" 
sheet = "QnA"  
df = pd.DataFrame(data=pd.read_excel(io=file_name, sheet_name=sheet), columns=["Type", "Question", "Answer"])

df.head(5)

,Type,Question,Answer
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 10:00 AM to 10:00 PM.
1,General Information,Is there free Wi-Fi available in Siam Paragon?,Free Wi-Fi available throughout the area of Si...
2,General Information,Are pets allowed in Siam Paragon?,"No, pets are not allowed in Siam Paragon."
3,General Information,What is the contact number for Siam Paragon's ...,Siam Paragon can be contacted at 02-610-8000 a...
4,General Information,Are there any prayer rooms available?,"Yes, there is a Muslim prayer room available o..."


In [22]:
test = df.head(5)

In [59]:
def query(row, client, model, index):
    prompt = row['Question']
    
    var_fn = {}
    
    index_result = query_index(client=client, model=model, index=index, prompt=prompt)
    var_fn.update({'index_result': index_result})
    
    file_ids = []
    for res in index_result:
        file_id = res.split(': ')[1]
        file_ids.append(file_id)
        
    contents = read_docs(file_ids=file_ids)
    
    response = generate_answer(client=client, prompt=prompt, contents=contents)
    var_fn.update({'response': response})
    
    answer = var_fn.get('response', '')
    answer = answer.replace('\r', '').replace('\n', '')
    answer = answer.replace('Answer: ', '')
   
    return answer

In [60]:
test['GPT'] = test.apply(query, args=(client, model, index), axis = 1)

/var/folders/yb/sc5y32796m90sprjzy0bmk9h0000gn/T/ipykernel_18886/1006263535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['GPT'] = test.apply(query, args=(client, model, index), axis = 1)


In [61]:
test

,Type,Question,Answer,GPT
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 10:00 AM to 10:00 PM.,The Bangkok Bank branch at Siam Paragon is ope...
1,General Information,Is there free Wi-Fi available in Siam Paragon?,Free Wi-Fi available throughout the area of Si...,"Yes, there is free Wi-Fi available in Siam Pa..."
2,General Information,Are pets allowed in Siam Paragon?,"No, pets are not allowed in Siam Paragon.","No, pets are not allowed inside Siam Paragon."
3,General Information,What is the contact number for Siam Paragon's ...,Siam Paragon can be contacted at 02-610-8000 a...,The contact number for Siam Paragon's main of...
4,General Information,Are there any prayer rooms available?,"Yes, there is a Muslim prayer room available o...","Yes, there are prayer rooms available on multi..."


In [63]:
df['GPT'] = df.apply(query, args=(client, model, index), axis = 1)

In [64]:
df

,Type,Question,Answer,GPT
0,General Information,When is Siam Paragon opening hours?,Siam Paragon opens from 10:00 AM to 10:00 PM.,The Bangkok Bank branch at Siam Paragon is ope...
1,General Information,Is there free Wi-Fi available in Siam Paragon?,Free Wi-Fi available throughout the area of Si...,"Yes, there is free Wi-Fi available in Siam Par..."
2,General Information,Are pets allowed in Siam Paragon?,"No, pets are not allowed in Siam Paragon.","No, pets are not allowed in Siam Paragon."
3,General Information,What is the contact number for Siam Paragon's ...,Siam Paragon can be contacted at 02-610-8000 a...,"Unfortunately, the information provided does n..."
4,General Information,Are there any prayer rooms available?,"Yes, there is a Muslim prayer room available o...","Yes, there are 3 prayer rooms available in Sia..."
...,...,...,...,...
103,Events and Promotion,Are there any ongoing sales or promotions?,The promotions and sales listed are exclusive ...,
104,Events and Promotion,How can I attend a Christmas show at Paragon?,While I don't have specific information on the...,"To attend a Christmas show at Paragon Hall, y..."
105,Events and Promotion,Is there a schedule for holiday events like Ch...,While I don't have specific information on cur...,The store does not mention any specific holid...
106,Events and Promotion,How can I get notified about upcoming promotions?,To stay informed about upcoming promotions at ...,There are a few ways to stay updated on upcom...


In [65]:
df.to_excel('response01.xlsx', index=False)